In [48]:
from google.cloud import aiplatform
import tensorflow as tf
import numpy as np

In [49]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query = """SELECT * FROM `ihr-vertex-pipelines.ihr_live_workshop.transactions` LIMIT 1"""
job = client.query(query)
df = job.to_dataframe()

In [50]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.75161,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.04315,-0.046401,0.0,0


In [51]:
tft_location = "gs://ihr-live-workshop/pipeline/237148598933/ihr-my-pipeline-20220301101234/Transform_-6056932480487784448/transform_graph/"

In [52]:
import tensorflow_transform as tft

In [53]:
transform = tft.TFTransformOutput(tft_location)

In [58]:
t = transform.transform_raw_features(d)

In [59]:
project = "ihr-vertex-pipelines"
region = "europe-west4"
aiplatform.init(project=project, location=region)

In [60]:
del t['Class']
t

{'V5': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.5474176]], dtype=float32)>,
 'V9': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.4482884]], dtype=float32)>,
 'V26': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.52740175]], dtype=float32)>,
 'V8': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.71421057]], dtype=float32)>,
 'V21': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.45735976]], dtype=float32)>,
 'V6': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.49328607]], dtype=float32)>,
 'V18': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.56166637]], dtype=float32)>,
 'V25': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.5324413]], dtype=float32)>,
 'V22': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.5886178]], dtype=float32)>,
 'V20': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.41340366]], dtype=float32)>,
 'V13': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.515

In [61]:
for k, v in t.items():
    t[k] = v.numpy().tolist()[0]

## Predicition using library and Google default credentials

In [62]:
t

{'V5': [0.5474175810813904],
 'V9': [0.4482884109020233],
 'V26': [0.5274017453193665],
 'V8': [0.714210569858551],
 'V21': [0.45735976099967957],
 'V6': [0.4932860732078552],
 'V18': [0.5616663694381714],
 'V25': [0.5324413180351257],
 'V22': [0.5886178016662598],
 'V20': [0.4134036600589752],
 'V13': [0.5156365633010864],
 'Amount': [0.0],
 'V12': [0.8216264247894287],
 'V2': [0.7502142190933228],
 'V1': [0.9430434107780457],
 'V16': [0.6249333620071411],
 'V14': [0.6819402575492859],
 'V23': [0.6696200966835022],
 'V24': [0.4815780818462372],
 'V28': [0.3397659659385681],
 'V10': [0.5074817538261414],
 'V19': [0.4424729645252228],
 'V17': [0.7333030700683594],
 'V11': [0.2604777216911316],
 'V7': [0.5057412385940552],
 'V27': [0.6487262845039368],
 'V15': [0.29382216930389404],
 'V3': [0.8279098272323608],
 'V4': [0.29084327816963196]}

In [20]:
endpoint_id = "7717252213034188800"
instances = {"instances": [t]} # Not for AI Platform lib!!

In [21]:
endpoint = aiplatform.Endpoint(endpoint_id)

In [29]:
endpoint.predict(instances=[t])

Prediction(predictions=[[0.000241786242]], deployed_model_id='3934325283066216448', explanations=None)

## Prediction using HTTP request and JSON tokens

In [30]:
import requests
import json

In [37]:
url = f"https://europe-west4-aiplatform.googleapis.com/v1/projects/{project}/locations/{region}/endpoints/{endpoint_id}:predict"

In [38]:
url

'https://europe-west4-aiplatform.googleapis.com/v1/projects/ihr-vertex-pipelines/locations/europe-west4/endpoints/7717252213034188800:predict'

In [34]:
!gcloud auth application-default print-access-token

ya29.A0ARrdaM_NBGckoS52wzMA-1nbNoVxTUXWiVIs2bGQql5tlVc2t5kRFAlvKWzyfeAR20XSHojb_DcBDXQmZnwMAKel9-F_T0JBENy8ey4SLNenECT9pETMQHdbhy6eyDWh096UkyO9McnYzAEd9gBLGMZka2j-nwVtzHKCvg


In [39]:
token = "ya29.A0ARrdaM_NBGckoS52wzMA-1nbNoVxTUXWiVIs2bGQql5tlVc2t5kRFAlvKWzyfeAR20XSHojb_DcBDXQmZnwMAKel9-F_T0JBENy8ey4SLNenECT9pETMQHdbhy6eyDWh096UkyO9McnYzAEd9gBLGMZka2j-nwVtzHKCvg"

In [41]:
body = json.dumps(instances)

In [42]:
body

'{"instances": [{"V28": [0.3397659659385681], "V12": [0.8216264247894287], "V3": [0.8279098272323608], "V5": [0.5474175810813904], "V19": [0.4424729645252228], "V17": [0.7333030700683594], "V23": [0.6696200966835022], "V22": [0.5886178016662598], "V6": [0.4932860732078552], "V4": [0.29084327816963196], "V24": [0.4815780818462372], "V7": [0.5057412385940552], "V27": [0.6487262845039368], "V11": [0.2604777216911316], "V16": [0.6249333620071411], "V21": [0.45735976099967957], "V2": [0.7502142190933228], "V14": [0.6819402575492859], "V18": [0.5616663694381714], "V9": [0.4482884109020233], "V15": [0.29382216930389404], "V1": [0.9430434107780457], "V13": [0.5156365633010864], "Amount": [0.0], "V20": [0.4134036600589752], "V10": [0.5074817538261414], "V26": [0.5274017453193665], "V8": [0.714210569858551], "V25": [0.5324413180351257]}]}'

In [44]:
header = {"Authorization": f"Bearer {token}", "Content-type": "application/json"}

In [46]:
o = requests.post(url, data=body, headers=header)

In [47]:
o.json()

{'predictions': [[0.000241786242]],
 'deployedModelId': '3934325283066216448',
 'model': 'projects/237148598933/locations/europe-west4/models/5227219015935459328',
 'modelDisplayName': 'v1646213728'}